In [1]:
# This will reload the imported modules (e.g. src.analysis, src.model) every time you execute the jupyter cells, so that you don't need to restart the notebook after updating the source codes.
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, SwitchTransformersForConditionalGeneration

tokenizer = AutoTokenizer.from_pretrained("google/switch-base-8")
model = SwitchTransformersForConditionalGeneration.from_pretrained("google/switch-base-8", )



input_ids = tokenizer(
    ("summarize: studies have shown that owning a dog is good for you", "translate English to German: Who is tushar krishna?"), return_tensors="pt", padding=True ).input_ids  # Batch size 1


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [3]:
!nvidia-smi

Mon Mar  4 19:07:03 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA TITAN RTX               On  | 00000000:1A:00.0 Off |                  N/A |
| 57%   76C    P2             279W / 280W |  21543MiB / 24576MiB |    100%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
input_ids

tensor([[21603,    10,  2116,    43,  2008,    24,   293,    53,     3,     9,
          1782,    19,   207,    21,    25,     1,     0,     0,     0],
        [13959,  1566,    12,  2968,    10,  2645,    19,     3,    17,   302,
          3272,     3, 10648,     7,   107,    29,     9,    58,     1]])

In [6]:
outputs = model.generate(input_ids, return_dict_in_generate=True, encoder_router_logits=False, decoder_router_logits=True)


In [7]:
for i,tokens_logit in enumerate(outputs.decoder_router_logits):
    print("Token:",i)
    for j, layer_logit in enumerate(tokens_logit):
        
        if j%2 == 1:
            print("\t Layer:", j)
            print(layer_logit)

Token: 0
	 Layer: 1
(tensor([[[ 6.6627, 14.7714, -5.9367,  1.8555,  0.9199, -4.8314,  0.4672,
          -4.8797]],

        [[ 7.0231, 14.8307, -5.7842,  1.4015,  0.3221, -4.9748,  0.7253,
          -4.3454]]]), tensor([[1],
        [1]]))
	 Layer: 3
(tensor([[[-1.2576,  0.0987, -2.7593,  0.0624, -0.6283, -0.5421,  4.7205,
          -0.3745]],

        [[-1.2581,  0.0993, -2.7617,  0.0598, -0.6274, -0.5478,  4.7195,
          -0.3779]]]), tensor([[6],
        [6]]))
	 Layer: 5
(tensor([[[-0.5804, -0.0034,  2.2951,  2.3270, -2.2949, -0.2171, -2.2005,
          -1.3653]],

        [[-0.5793,  0.0035,  2.3047,  2.3291, -2.2836, -0.2030, -2.1879,
          -1.3564]]]), tensor([[3],
        [3]]))
	 Layer: 7
(tensor([[[ 1.3390, -1.0821, -2.6922,  8.2390,  3.3150, -1.7004, -0.1692,
           8.2398]],

        [[ 1.3743, -1.0610, -2.6781,  8.2605,  3.3239, -1.7129, -0.1850,
           8.2983]]]), tensor([[7],
        [7]]))
	 Layer: 9
(tensor([[[ 23.3037, -10.8031,   2.1552, -13.4547,  -5.0

In [27]:
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("cais/mmlu", 'anatomy')

/nethome/abambhaniya3/.conda/envs/moe_scheduler/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for cais/mmlu contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cais/mmlu
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Generating dev split: 100%|██████████| 5/5 [00:00<00:00, 86.10 examples/s]


In [29]:
dataset['auxiliary_train']

Dataset({
    features: ['question', 'subject', 'choices', 'answer'],
    num_rows: 99842
})

In [30]:
from langchain.prompts import PromptTemplate


ModuleNotFoundError: No module named 'langchain'